<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.883 · Aprendizaje por refuerzo</p>
<p style="margin: 0; text-align:right;">2021-1 · Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# PEC 1 - Soluciones tabulares

En esta práctica implementaremos los diferentes métodos de aprendizaje por refuerzo vistos en las Partes I y II del curso. En concreto, nos centraremos en la definición de un entorno e implementaremos los diferentes métodos para buscar una solución óptima del problema.

**<u>Importante</u>**: 

1. Esta PEC debe realizarse de **forma estrictamente individual**. Cualquier indicio de copia será penalizado con un suspenso (D) para todas las partes implicadas y la posible evaluación negativa de la asignatura de forma íntegra.

2. Es necesario que el estudiante indique **todas las fuentes que ha utilizado para la realización de la PEC**. De no ser así, se considerará que el estudiante ha cometido plagio, siendo penalizado con un suspenso (D) y la posible evaluación negativa de la asignatura de forma íntegra.

3. La entrega debe hacerse en **formato notebook y en formato html** donde se vea el código, los resultados y comentarios de cada ejercicio. Es decir, deben entregarse dos ficheros: uno con extensión .ipynb y otro .html. Para exportar el notebook a html puede hacerse desde el menú File $\to$ Download as $\to$ HTML.


## 0. El entorno __CliffWalking__

El entorno __CliffWalking__ consiste en un agente que se mueve en una cuadrícula de dimensiones 12x4 (ancho x alto). En cada paso, el agente tiene 4 opciones de acción o movimiento: ARRIBA, DERECHA, ABAJO, IZQUIERDA. La posición de cada casilla viene dada por una pareja de números naturales [x, y], donde la posición de la esquina de arriba a la izquierda sería el origen de coordenadas [0, 0]. El agente siempre sale de la misma casilla [0, 3] (esquina abajo izquierda) y el juego termina cuando el agente llega a la casilla de llegada [11, 3] (esquina abajo derecha).

El entorno se corresponde con el ejemplo 'Cuadrícula con precipicio' explicado en la sección 3.2.1. del módulo "Métodos de Diferencia Temporal". El problema radica en que en todas las casillas la recompensa inmediata es R=-1 excepto en las casillas que unen en línea recta la casilla de salida con la de llegada, casillas  [1, 3] a  [10, 3]. En estas casillas, que simulan un precipicio, la recompensa es R=-100 y se vuelve a la casilla inicial.

<img src="CliffWalking.png">
Fuente: Sutton & Barto, 2018

El código para implementar este entorno, que se encuentra disponible en el fichero adjunto `cliff_env.py`, ha sido adaptado del siguiente enlace:
https://pypi.org/project/gym-gridworlds/

Vamos a empezar cargando el entorno y ver qué características tiene, ejecutando un episodio de prueba.

### 0.1. Carga de datos
El siguiente código carga los paquetes necesarios para ejecutar el ejemplo, crea el entorno mediante la instanciación de un objeto de la clase `CliffEnv` (importada del archivo adjunto `cliff_env.py`) e imprime por pantalla la dimensión del espacio de acciones (0=arriba, 1=derecha, 2=abajo y 3=izquierda), el espacio de observaciones (una tupla que indica la posición del agente en la cuadrícula) y el rango de la variable de recompensa (cuyo valor es -1 o -100 dependiendo de la casilla visitada y que por tanto va de menos infinito a más infinito).

In [25]:
!pip install gym

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [26]:
import gym
import numpy as np
import cliff_env as cenv

env=cenv.CliffEnv()
print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))
print("Reward range is {} ".format(env.reward_range))

Action space is Discrete(4) 
Observation space is Tuple(Discrete(12), Discrete(4)) 
Reward range is (-inf, inf) 


### 0.2. Ejecución de un episodio

A continuación, realizaremos la ejecución de un episodio del entorno CliffWalking utilizando un agente que selecciona las acciones de forma aleatoria.


In [27]:
# Inicializamos el entorno
obs = env.reset()
t, total_reward, done = 0, 0, False

print("Obs inicial: {} ".format(obs))

switch_action = {
        0: "U",
        1: "R",
        2: "D",
        3: "L",
    }

while not done:
    
    # Elegir una acción aleatoria (ésta es la implementación del agente)
    action = env.action_space.sample()
    
    # Ejecutar la acción y esperar la respuesta del entorno
    new_obs, reward, done, info = env.step(action)
    
    # Imprimir time-step
    print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], new_obs, reward))
    
    # Actualizar variables
    obs = new_obs
    total_reward += reward
    t += 1
    
print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
env.close()

Obs inicial: (0, 3) 
Action: D -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: L -> Obs: (0, 0) and reward: -1
Action: R -> Obs: (1, 0) and reward: -1
Action: R -> Obs: (2, 0) and reward: -1
Action: D -> Obs: (2, 1) and reward: -1
Action: R -> Obs: (3, 1) and reward: -1
Action: U -> Obs: (3, 0) and reward: -1
Action: U -> Obs: (3, 0) and reward: -1
Action: D -> Obs: (3, 1) and reward: -1
Action: D -> Obs: (3, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Ob

Action: L -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: R -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and rewar

Action: R -> Obs: (4, 1) and reward: -1
Action: U -> Obs: (4, 0) and reward: -1
Action: R -> Obs: (5, 0) and reward: -1
Action: L -> Obs: (4, 0) and reward: -1
Action: L -> Obs: (3, 0) and reward: -1
Action: R -> Obs: (4, 0) and reward: -1
Action: D -> Obs: (4, 1) and reward: -1
Action: R -> Obs: (5, 1) and reward: -1
Action: U -> Obs: (5, 0) and reward: -1
Action: R -> Obs: (6, 0) and reward: -1
Action: L -> Obs: (5, 0) and reward: -1
Action: D -> Obs: (5, 1) and reward: -1
Action: R -> Obs: (6, 1) and reward: -1
Action: U -> Obs: (6, 0) and reward: -1
Action: L -> Obs: (5, 0) and reward: -1
Action: R -> Obs: (6, 0) and reward: -1
Action: R -> Obs: (7, 0) and reward: -1
Action: D -> Obs: (7, 1) and reward: -1
Action: D -> Obs: (7, 2) and reward: -1
Action: U -> Obs: (7, 1) and reward: -1
Action: L -> Obs: (6, 1) and reward: -1
Action: D -> Obs: (6, 2) and reward: -1
Action: L -> Obs: (5, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -100
Action: L -> Obs: (0, 3) and reward: -

Action: R -> Obs: (2, 1) and reward: -1
Action: L -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: U -> Obs: (1, 0) and reward: -1
Action: D -> Obs: (1, 1) and reward: -1
Action: D -> Obs: (1, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: U -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: R -> Obs: (2, 2) and reward: -1
Action: L -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward:

Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: R -> Obs: (0, 3) and reward: -100
Action: L -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: U -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and rewar

Action: L -> Obs: (2, 2) and reward: -1
Action: R -> Obs: (3, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -100
Action: L -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: U -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: U -> Obs: (1, 1) and reward: -1
Action: D -> Obs: (1, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -100
Action: U -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: L -> Obs: (0, 0) and reward: -1
Action: U -> Obs: (0, 0) and rewar

Action: U -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: D -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: R -> Obs: (0, 3) and reward: -100
Action: U -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: U -> Obs: (1, 1) and reward: -1
Action: D -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (0, 3) and reward: -100
Action: L -> Obs: (0, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: U -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and rew

## 1. Modificación del entorno (1,5 puntos)
El entorno CliffWalking tiene varios parámetros que pueden ser modificados:


*   La dimensión de la cuadrícula.
*   La posición del precipicio.
*   La posición de las casillas de salida y de llegada.
*   La recompensa inmediata.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 1.1</strong> (0,75 puntos)</div>

Modificar el codigo de CliffWalking (fichero adjunto `cliff_env.py`) para que represente las propiedades de la cuadrícula descritas a continuación.

- Cuadrícula 5x11
- Posición del precipicio = Casillas [1,0], [2,0], [3,0], [4,0], [5,0], [6,0], [7,0], [8,0] y [9,0]
- Casilla de inicio [0,0]
- Casilla final [10,0]
- Recompensa en casillas normales = -1
- Recompensa en casillas del precipicio = -500 (y volvemos a la casilla de salida)

Guardar el entorno modificado en el archivo `cliff_env_v2.py`, en la misma carpeta que el original. Este archivo debe entregarse junto a los archivos de este notebook (.ipynb y .html).

In [28]:
import gym
import numpy as np
import cliff_env_v2 as ce2

env=ce2.CliffEnv()
print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))
print("Reward range is {} ".format(env.reward_range))

Action space is Discrete(4) 
Observation space is Tuple(Discrete(11), Discrete(5)) 
Reward range is (-inf, inf) 


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 1.2</strong> (0,25 puntos)</div>

A continuación, implementar un agente que lleve a cabo una política aleatoria. Comprobar que las casillas visitadas y las recompensas recibidas se corresponden con las acciones y el entorno programado.

Mostrar la trayectoria seguida por el agente. No es necesario graficarla, tan sólo mostrar las coordenadas de las casillas visitadas en orden y las recompensas recibidas.

In [29]:
# Environment reset
obs = env.reset()
t, total_reward, done = 0, 0, False

print("Obs inicial: {} ".format(obs))

switch_action = {
        0: "U",
        1: "R",
        2: "D",
        3: "L"
    }

Obs inicial: (0, 0) 


In [30]:
while not done:
    
    # Elegir una acción aleatoria (ésta es la implementación del agente)
    action = env.action_space.sample()
    
    # Ejecutar la acción y esperar la respuesta del entorno
    new_obs, reward, done, info = env.step(action)
    
    # Imprimir time-step
    print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], new_obs, reward))
    
    # Actualizar variables
    obs = new_obs
    total_reward += reward
    t += 1
    
print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
env.close()

Action: D -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: R -> Obs: (2, 2) and reward: -1
Action: L -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: R -> Obs: (1, 2) and reward: -1
Action: U -> Obs: (1, 1) and reward: -1
Action: D -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 4) and reward: -1
Action: R -> Obs: (1, 4) and reward: -1
Action: R -> Obs: (2, 4) and reward: -1
Action: R -> Obs: (3, 4) and reward: -1
Action: U -> Obs: (3, 3) and reward: -1
Action: U -> Obs: (3, 2) and reward: -1
Action: L -> Obs: (2, 2) and reward: -1


Action: R -> Obs: (1, 2) and reward: -1
Action: R -> Obs: (2, 2) and reward: -1
Action: D -> Obs: (2, 3) and reward: -1
Action: L -> Obs: (1, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (1, 3) and reward: -1
Action: R -> Obs: (2, 3) and reward: -1
Action: R -> Obs: (3, 3) and reward: -1
Action: L -> Obs: (2, 3) and reward: -1
Action: L -> Obs: (1, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (1, 3) and reward: -1
Action: D -> Obs: (1, 4) and reward: -1
Action: U -> Obs: (1, 3) and reward: -1
Action: D -> Obs: (1, 4) and reward: -1
Action: U -> Obs: (1, 3) and reward: -1
Action: R -> Obs: (2, 3) and reward: -1
Action: L -> Obs: (1, 3) and reward: -1
Action: U -> Obs: (1, 2) and reward: -1
Action: R -> Obs: (2, 2) and reward: -1
Action: R -> Obs: (3, 2) and reward: -1
Action: R -> Obs: (4, 2) and reward: -1
Action: D -> Obs: (4, 3) and reward: -1
Action: R -> Obs: (5, 3) and reward: -1
Action: D -> Obs: (5, 4) and reward: -1


Action: R -> Obs: (0, 0) and reward: -500
Action: L -> Obs: (0, 0) and reward: -1
Action: R -> Obs: (0, 0) and reward: -500
Action: D -> Obs: (0, 1) and reward: -1
Action: U -> Obs: (0, 0) and reward: -1
Action: D -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: L -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: R -> Obs: (2, 1) and reward: -1
Action: R -> Obs: (3, 1) and reward: -1
Action: D -> Obs: (3, 2) and reward: -1
Action: L -> Obs: (2, 2) and reward: -1
Action: L -> Obs: (1, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: U -> Obs: (0, 2) and reward: -1
Action: L -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 4) and reward: -1
Action: U -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (1, 3) and reward: -1
Action: L -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 4) and reward:

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 1.3</strong> (0,5 puntos)</div>

A continuación, implementar un agente que lleve a cabo la política óptima determinista, es decir, que partiendo de la casilla inicial [0,0] llegue a la casilla final [10,0] en el menor número de pasos posible. ¿Cuál es el valor del número de pasos mínimos?

Mostrar la trayectoria seguida por el agente y el retorno obtenido. No es necesario graficarla, tan sólo mostrar las coordenadas de las casillas visitadas en orden.

In [31]:
# Environment reset
obs = env.reset()
t, total_reward, done = 0, 0, False

print("Obs inicial: {} ".format(obs))

switch_action = {
        0: "U",
        1: "R",
        2: "D",
        3: "L"
    }

Obs inicial: (0, 0) 


In [32]:
new_obs, reward, done, info = env.step(2)
# Imprimir time-step
print("Action: {} -> Obs: {} and reward: {}".format(switch_action[2], new_obs, reward))

# Actualizar variables
obs = new_obs
total_reward += reward
t += 1
while new_obs[0] != 10:
    new_obs, reward, done, info = env.step(1)
    # Imprimir time-step
    print("Action: {} -> Obs: {} and reward: {}".format(switch_action[1], new_obs, reward))

    # Actualizar variables
    obs = new_obs
    total_reward += reward
    t += 1
    
new_obs, reward, done, info = env.step(0)
# Imprimir time-step
print("Action: {} -> Obs: {} and reward: {}".format(switch_action[0], new_obs, reward))

# Actualizar variables
obs = new_obs
total_reward += reward
t += 1


Action: D -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: R -> Obs: (2, 1) and reward: -1
Action: R -> Obs: (3, 1) and reward: -1
Action: R -> Obs: (4, 1) and reward: -1
Action: R -> Obs: (5, 1) and reward: -1
Action: R -> Obs: (6, 1) and reward: -1
Action: R -> Obs: (7, 1) and reward: -1
Action: R -> Obs: (8, 1) and reward: -1
Action: R -> Obs: (9, 1) and reward: -1
Action: R -> Obs: (10, 1) and reward: -1
Action: U -> Obs: (10, 0) and reward: -1


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentarios:</strong>
<br><br>
Escribir aquí los comentarios oportunos
</div>


## 2. Programación Dinámica (2.5 puntos)

Dado que el entorno es determinista, es factible encontrar una política óptima (que en este caso es única) que consiga el mayor retorno (y por tanto la trayectoria más corta).

El objetivo de este apartado es realizar una estimación de la política óptima mediante los métodos de Programación Dinámica, en concreto estudiaremos el algoritmo *Iteración del valor*. 

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 2.1</strong> (1 puntos)</div>

Implementar el Algoritmo 3 explicado en el módulo "Programación Dinámica" utilizando los siguientes parámetros:
    
- Theta = 0.001
- Factor de descuento (*discount factor*) = 1

In [33]:
# Environment reset
obs = env.reset()
t, total_reward, done = 0, 0, False

print("Obs inicial: {} ".format(obs))

# Escribir el código aquí

#Define all states
states = []
for i in range(11):
    for j in range(5):
        states.append((i,j))

# initialize policy
policy={}
for s in states:
    policy[s] = env.action_space.sample()

# initialize V(s)
V = {}
for s in states:
    V[s] = 0

Obs inicial: (0, 0) 


In [34]:
float('-inf')

-inf

In [35]:
def dp_iteration_value(theta=0.001, discount=1):
    THETA = theta
    DISCOUNT_FACTOR = discount
    converged = False
    i = 0
    while not converged:
        DELTA = 0
        for s in states:
            if s != (10,0):
                oldV = V[s]
                newV = float('-inf')
                for action in range(env.action_space.n):
                    env.S = s
                    new_obs, reward, done, info = env.step(action)
                    currentActV = (reward + V[new_obs] * DISCOUNT_FACTOR)
                    if currentActV > newV:
                        newV = currentActV
                        policy[s] = action
                V[s] = newV
                DELTA = max(DELTA, np.abs(oldV - V[s]))
        if DELTA <= THETA:
            converged = True
        i += 1
    return V,policy,i

In [36]:
V,policy,i = dp_iteration_value()

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 2.2</strong> (0.5 puntos)</div>

Implementar una función que imprima por pantalla los valores finales de la función de valor para cada celda. 

Se muestra a continuación una imagen de ejemplo para la cuadrícula del entorno modificado de 11x5.

<img src="ValueFunction.png">

In [37]:
# función que muestra los valores de la función de estado V(s) en la cuadricula

def print_values(V, width, height):
    for i in range(height):
        print('--------------------------------------------------')
        string = ''
        for j in range(width):
            string = string +  (str(round(V[(j,i)], 2)) + str (' |'))
        print(string)
        if i == 4:
            print('--------------------------------------------------')

In [38]:
print("values:")
print_values(V, 11, 5)

values:
--------------------------------------------------
-12 |-11 |-10 |-9 |-8 |-7 |-6 |-5 |-4 |-1 |0 |
--------------------------------------------------
-11 |-10 |-9 |-8 |-7 |-6 |-5 |-4 |-3 |-2 |-1 |
--------------------------------------------------
-12 |-11 |-10 |-9 |-8 |-7 |-6 |-5 |-4 |-3 |-2 |
--------------------------------------------------
-13 |-12 |-11 |-10 |-9 |-8 |-7 |-6 |-5 |-4 |-3 |
--------------------------------------------------
-14 |-13 |-12 |-11 |-10 |-9 |-8 |-7 |-6 |-5 |-4 |
--------------------------------------------------


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 2.3</strong> (0.5 puntos)</div>

Implementar una función que imprima por pantalla la política óptima encontrada para cada celda. 

Se muestra a continuación una imagen de ejemplo para la cuadrícula del entorno modificado de 11x5.

<img src="Policy.png">

In [39]:
def print_policy_DP(policy, width, height):
    for i in range(height):
        print('----------------------')
        string = ''
        for j in range(width):
            string = string + switch_action[policy[(j,i)]] + str ('|')
        print(string)
        if i == 4:
            print('----------------------')

In [40]:
print("policy:")
print_policy_DP(policy,11,5)

policy:
----------------------
D|D|D|D|D|D|D|D|D|R|R|
----------------------
R|R|R|R|R|R|R|R|R|R|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 2.4</strong> (0.5 puntos)</div>

Ejecutar un episodio con la política óptima encontrada y mostrar la trayectoria del agente y el retorno obtenido.

In [41]:
def execute_episode_DP(policy,env):
    obs = env.reset()
    t, total_reward, done = 0, 0, False
    while not done:
        #Elegimos la acción que indica la política para el estado actual
        action = policy[obs]

         # Ejecutar la acción y esperar la respuesta del entorno
        new_obs, reward, done, info = env.step(action)

        # Imprimir time-step
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], new_obs, reward))

        # Actualizar variables
        obs = new_obs
        total_reward += reward
        t += 1
    
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()

In [42]:
execute_episode_DP(policy,env)

Action: D -> Obs: (0, 1) and reward: -1
Action: R -> Obs: (1, 1) and reward: -1
Action: R -> Obs: (2, 1) and reward: -1
Action: R -> Obs: (3, 1) and reward: -1
Action: R -> Obs: (4, 1) and reward: -1
Action: R -> Obs: (5, 1) and reward: -1
Action: R -> Obs: (6, 1) and reward: -1
Action: R -> Obs: (7, 1) and reward: -1
Action: R -> Obs: (8, 1) and reward: -1
Action: R -> Obs: (9, 1) and reward: -1
Action: R -> Obs: (10, 1) and reward: -1
Action: U -> Obs: (10, 0) and reward: -1
Episode finished after 12 timesteps and reward was -12 


## 3. Métodos de Montecarlo (2 puntos)

El objetivo de este apartado es realizar una estimación de la política óptima mediante los métodos de Montecarlo, en concreto estudiaremos el algoritmo *On-policy every-visit MC control (para políticas $\epsilon$-soft)*. 

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 3.1</strong> (1 punto)</div>

A partir del Algoritmo 3 explicado en el módulo "Métodos de Montecarlo", modificarlo para implementar el algoritmo *On-policy every-visit MC control (para políticas $\epsilon$-soft)* utilizando los siguientes parámetros:
    
- Número de episodios = 100000
- Epsilon inicial = 1
- Factor de decaimiento de epsilon (*epsilon decay*) = 0.999
- Actualizar epsilon según: $$\textrm{max}(\epsilon · \epsilon_{\textrm{decay}}, 0.01)$$
- Factor de descuento = 1

In [43]:
# Environment reset
obs = env.reset()
t, total_reward, done = 0, 0, False

In [45]:
from collections import defaultdict
import sys
def make_epsilon_greedy_policy(Q, epsilon, num_Actions):
    """
    Crea una política epsilon-greedy basada en una función de valor de acción Q y epsilon
    
    Args:
        Q: Un diccionario cuya correspondencia es state -> action-values.
           Cada valor es un array de numpy de longitud num_Actions (see below)
        epsilon: La probabilidad de seleccionar una acción aleatoria (float entre 0 and 1).
        num_Actions: Número de acciones del entorno. (en el caso del WIndyGridWorld es 4)
    
    Returns:
        Una función que tome como argumento la observación y devuelva como resultado
        las probabilidades de cada acción como un array de numpy de longitud num_Actions.
    """
    def policy_fn(observation):

        A = np.ones(num_Actions, dtype=float) * epsilon / num_Actions
        best_action = np.argmax(list((Q[observation].values())))
        A[best_action] += (1.0 - epsilon)

        return A
    
    return policy_fn

In [46]:
def mc_control_on_policy_epsilon_greedy(env, num_episodes, discount=1.0, epsilon=1, epsilon_decay=0.999):
    """
    Control mediante métodos de Montecarlo usando políticas Epsilon-Greedy
    Encuentra una política epsilon-greedy.
    
    Args:
        env: entorno OpenAI gym.
        num_episodes: Número de episodios de la muestra.
        discount: factor de descuento.
        epsilon: La probabilidad de seleccionar una acción aleatoria (float entre 0 and 1)
    
    Returns:
        Una tupla (Q, policy).
        Q: Un diccionario cuya correspondencia es state -> action-values.
        policy: Una función que toma como argumento la observación y devuelve como resultado
                las probabilidades de cada acción
    """
    
    # Almacenamos la suma y el número de retornos de cada estado para calcular
    # el promedio. Podríamos usar un array para guardar todos los retornos
    # (como en el libro) pero es ineficiente en términos de memoria.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # La función de valor de acción Q.
    # Un diccionario anidado cuya correspondencia es state -> (action -> action-value).
    # Inicialmente la inicializamos a cero
    Q = {}

    for s in states:
        Q[s] = {}
        for a in range(env.action_space.n):
            Q[s][a] = np.random.rand()
    
    # La política que estamos siguiendo
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(1, num_episodes + 1):
        # Imprimimos en qué episodio estamos, útil para debugar.
        if i_episode % 10 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        # Generamos un episodio y lo almacenamos
        # Un episodio es un array de las tuplas (state, action, reward)
        episode = []
        state = env.reset()
        done = False
        while not done:
            probs = policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state

        # Encontramos todos los pares (estado, acción) que hemos visitado en este episodio
        # Convertimos cada estado en una tupla para poder usarlo como clave del diccionario
        sa_in_episode = set([(tuple(x[0]), x[1]) for x in episode])
        for state, action in sa_in_episode:
            sa_pair = (state, action)
            # Sumamos todas las recompensas
            if (discount != 1):
                G = sum([x[2]*(discount**i) for i,x in enumerate(episode)])
            else:
                G = sum([x[2] for i,x in enumerate(episode)])
            # Calculamos el retorno promedio para este estado en todos los episodios muestreados
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0
            Q[state][action] = returns_sum[sa_pair] / returns_count[sa_pair]
        
        # La política se mejora implícitamente al ir cambiando los valores de Q
        epsilon = max(epsilon * epsilon_decay, 0.01)
        policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    return Q, policy

In [21]:
Q, policy = mc_control_on_policy_epsilon_greedy(env, num_episodes=100000)

Episode 100000/100000.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 3.2</strong> (0.5 puntos)</div>

Implementar una función que imprima por pantalla la política óptima encontrada para cada celda. 

In [22]:
def print_policy_MC(policy, width, height):
    switch_action = {
        0: "U",
        1: "R",
        2: "D",
        3: "L"
    }
    for i in range(height):
        print("------------------------------------------------------------------------------------------")
        for j in range(width):
            arr = np.array(policy((j,i)))
            act = int(np.where(arr == np.amax(arr))[0])
            a = switch_action[act]
            print("  %s  |" % a, end="")
        print("")

In [23]:
print_policy_MC(policy,11,5)

------------------------------------------------------------------------------------------
  D  |  D  |  U  |  U  |  R  |  U  |  D  |  L  |  D  |  L  |  D  |
------------------------------------------------------------------------------------------
  D  |  D  |  D  |  D  |  L  |  R  |  D  |  R  |  R  |  R  |  U  |
------------------------------------------------------------------------------------------
  D  |  D  |  R  |  R  |  D  |  D  |  D  |  U  |  D  |  R  |  U  |
------------------------------------------------------------------------------------------
  D  |  D  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |  U  |
------------------------------------------------------------------------------------------
  R  |  R  |  U  |  U  |  R  |  R  |  R  |  R  |  U  |  U  |  U  |


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 3.3</strong> (0.5 puntos)</div>

Ejecutar un episodio con la política óptima encontrada y mostrar la trayectoria del agente y el retorno obtenido.

In [24]:
def execute_episode_MC(policy, env):
    obs = env.reset()
    t, total_reward, done = 0, 0, False
    print("Obs inicial: {} ".format(obs))

    switch_action = {
            0: "U",
            1: "R",
            2: "D",
            3: "L",
        }

    while not done: # limitamos el número de time-steps de cada episodio a 1000
        
        # Elegimos la política óptima en cada caso (el máximo de la política Epsilon-Greedy)
        arr = np.array(policy(obs))
        action = arr.argmax()
    
        # Ejecutamos la acción y esperamos la respuesta del entorno
        new_obs, reward, done, info = env.step(action)
        obs = new_obs
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], obs, reward))
            
        total_reward += reward
        
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()

In [25]:
execute_episode_MC(policy,env)

Obs inicial: (0, 0) 
Action: D -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: D -> Obs: (0, 4) and reward: -1
Action: R -> Obs: (1, 4) and reward: -1
Action: R -> Obs: (2, 4) and reward: -1
Action: U -> Obs: (2, 3) and reward: -1
Action: R -> Obs: (3, 3) and reward: -1
Action: R -> Obs: (4, 3) and reward: -1
Action: R -> Obs: (5, 3) and reward: -1
Action: R -> Obs: (6, 3) and reward: -1
Action: R -> Obs: (7, 3) and reward: -1
Action: R -> Obs: (8, 3) and reward: -1
Action: R -> Obs: (9, 3) and reward: -1
Action: U -> Obs: (9, 2) and reward: -1
Action: R -> Obs: (10, 2) and reward: -1
Action: U -> Obs: (10, 1) and reward: -1
Action: U -> Obs: (10, 0) and reward: -1
Episode finished after 0 timesteps and reward was -18 


## 4. Métodos de Diferencia Temporal (2 puntos)

El objetivo de este apartado es realizar una estimación de la política óptima mediante los métodos de Diferencia Temporal en el entorno CliffWalking modificado anteriormente. Concretamente nos centraremos en el método SARSA.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 4.1</strong> (1 punto)</div>

Implementar el algoritmo *SARSA* explicado en el modulo "Aprendizaje por Diferencia Temporal" utilizando los siguientes parámetros:
    
- Número de episodios = 1000
- *learning rate* = 0.5
- *discount factor* = 1
- *epsilon* = 0.05  

In [47]:
#Function to choose the next action
def choose_action(state, epsilon, Q):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(list((Q[state].values())))
    return action

In [48]:
def sarsa_control_algorithm(epsilon = 0.05, total_episodes=1000, learning_rate=0.5, discount=1):
    #Initializing the Q-matrix
    Q = {}

    for s in states:
        Q[s] = {}
        for a in range(env.action_space.n):
            Q[s][a] = np.random.rand()
    for a in range(env.action_space.n):
        Q[(10,0)][a] = 0
    
    import sys 
    for episode in range(total_episodes+1):
        if episode % 10 == 0:
            print("\rEpisode {}/{}.".format(episode, total_episodes), end="")
            sys.stdout.flush()
        state1 = env.reset()
        action1 = choose_action(state1, epsilon, Q)
        done = False
        while not done:

            #Getting the next state
            state2, reward, done, info = env.step(action1)

            if not done:
                #Choosing the next action
                action2 = choose_action(state2, epsilon, Q)

                #Learning the Q-value
                target = reward + discount * Q[state2][action2]
                Q[state1] [action1] = Q[state1][action1] + learning_rate * (target - Q[state1][action1])

                state1 = state2
                action1 = action2
            else:
                target = reward + discount * 0
                Q[state1] [action1] = Q[state1][action1] + learning_rate * (target - Q[state1][action1])
                state1 = state2
                action1 = action2
    return Q


In [28]:
Q = sarsa_control_algorithm()

Episode 1000/1000.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 4.2</strong> (0.25 puntos)</div>

Mostrar por pantalla los valores Q estimados para cada par estado-acción.

In [29]:
Q

{(0, 0): {0: -26.549374227614386,
  1: -520.3998376030675,
  2: -18.210416948805907,
  3: -22.96597162357446},
 (0, 1): {0: -34.18311577207665,
  1: -20.022186688432235,
  2: -17.33037657772076,
  3: -19.443385810933265},
 (0, 2): {0: -19.327209579338476,
  1: -17.738810851997755,
  2: -16.53704547542055,
  3: -17.984674159894283},
 (0, 3): {0: -17.78944448326532,
  1: -15.70865772344795,
  2: -16.40572811082142,
  3: -16.429152471641927},
 (0, 4): {0: -15.997790639956204,
  1: -15.248927841905374,
  2: -16.525909188842604,
  3: -16.410922522419494},
 (1, 0): {0: 0.2183494028875771,
  1: 0.5902113523529913,
  2: 0.2780840309034852,
  3: 0.31825532683262203},
 (1, 1): {0: -384.5457754445262,
  1: -71.67621787727967,
  2: -17.980997817589177,
  3: -18.319373420187837},
 (1, 2): {0: -17.25073095508477,
  1: -15.68978448003614,
  2: -16.124389657424167,
  3: -18.002974079171246},
 (1, 3): {0: -15.631392884554135,
  1: -15.56544819908251,
  2: -14.864970986074505,
  3: -16.060403972774793},

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 4.3</strong> (0.25 puntos)</div>

Mostrar por pantalla los valores de la función de valor $v_\pi(s)$ estimada para cada estado.

In [30]:
# función que muestra los valores de la función de estado V(s) en la cuadricula

def print_values_SARSA(Q, width, height):
    for i in range(height):
        print("--------------------------------------------------------------------------")
        for j in range(width):
            a= np.argmax(list((Q[(j,i)].values())))
            policy[(j,i)] = a
            V[(j,i)] = round(Q[(j,i)][a],2)
            print("%s|" % V[(j,i)], end="")
        print("")

In [31]:
policy = {}
V = {}
print_values_SARSA(Q, 11, 5)

--------------------------------------------------------------------------
-18.21|0.59|0.64|0.61|0.9|0.51|0.92|0.99|0.84|0.98|0|
--------------------------------------------------------------------------
-17.33|-17.98|-14.93|-14.82|-6.39|-11.03|-11.04|-11.08|-3.0|-2.02|-1.0|
--------------------------------------------------------------------------
-16.54|-15.69|-14.7|-14.18|-11.97|-11.5|-11.27|-11.07|-9.96|-3.35|-2.13|
--------------------------------------------------------------------------
-15.71|-14.86|-13.57|-13.32|-12.16|-11.69|-11.51|-10.85|-10.24|-4.36|-3.44|
--------------------------------------------------------------------------
-15.25|-14.22|-13.77|-13.23|-12.43|-11.35|-9.93|-8.77|-7.12|-5.3|-4.51|


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 4.4</strong> (0.25 puntos)</div>

Implementar una función que imprima por pantalla la política óptima encontrada para cada celda. 

In [51]:
def print_policy_SARSA(Q, width, height):
    for i in range(height):
        print("-------------------------------------------------------------------")
        for j in range(width):
            a= np.argmax(list((Q[(j,i)].values())))
            policy[(j,i)] = a
            V[(j,i)] = round(Q[(j,i)][a],2)
            print("  %s  |" % switch_action[policy[(j,i)]], end="")
        print("")

In [33]:
policy = {}
V = {}
print_policy_SARSA(Q,11,5)

-------------------------------------------------------------------
  D  |  R  |  D  |  D  |  L  |  U  |  D  |  L  |  R  |  R  |  U  |
-------------------------------------------------------------------
  D  |  D  |  D  |  L  |  R  |  D  |  L  |  L  |  R  |  R  |  U  |
-------------------------------------------------------------------
  D  |  R  |  D  |  D  |  R  |  R  |  U  |  U  |  D  |  R  |  U  |
-------------------------------------------------------------------
  R  |  D  |  D  |  R  |  U  |  R  |  D  |  R  |  U  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 4.5</strong> (0.25 puntos)</div>

Ejecutar un episodio siguiendo la política óptima encontrada, donde se pueda reconocer la trayectoria seguida por el agente.

In [52]:
# ejecución de un episodio siguiendo la politica optima
def execute_episode_SARSA(q, env):
    obs = env.reset()
    t, total_reward, done = 0, 0, False
    print("Obs inicial: {} ".format(obs))

    switch_action = {
            0: "U",
            1: "R",
            2: "D",
            3: "L",
        }
    
    while not done:
        action = np.argmax(list((Q[obs].values())))
        
        # Ejecutar la acción y esperar la respuesta del entorno
        new_obs, reward, done, info = env.step(action)

        # Imprimir time-step
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], new_obs, reward))

        # Actualizar variables
        obs = new_obs
        total_reward += reward
        t += 1
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()


In [35]:
execute_episode_SARSA(Q, env)

Obs inicial: (0, 0) 
Action: D -> Obs: (0, 1) and reward: -1
Action: D -> Obs: (0, 2) and reward: -1
Action: D -> Obs: (0, 3) and reward: -1
Action: R -> Obs: (1, 3) and reward: -1
Action: D -> Obs: (1, 4) and reward: -1
Action: R -> Obs: (2, 4) and reward: -1
Action: R -> Obs: (3, 4) and reward: -1
Action: R -> Obs: (4, 4) and reward: -1
Action: R -> Obs: (5, 4) and reward: -1
Action: R -> Obs: (6, 4) and reward: -1
Action: R -> Obs: (7, 4) and reward: -1
Action: R -> Obs: (8, 4) and reward: -1
Action: R -> Obs: (9, 4) and reward: -1
Action: R -> Obs: (10, 4) and reward: -1
Action: U -> Obs: (10, 3) and reward: -1
Action: U -> Obs: (10, 2) and reward: -1
Action: U -> Obs: (10, 1) and reward: -1
Action: U -> Obs: (10, 0) and reward: -1
Episode finished after 18 timesteps and reward was -18 


## 5. Comparativa de los algoritmos (2 puntos)

En este apartado haremos diferentes comparativas de las implementaciones de los metodos programados en los apartados anteriores.

Compararemos el comportamiento de los algoritmos al modificar los valores del factor de descuento, el *learning rate* y el numero de episodios.

Para cada ejercicio se debe mostrar y justificar el resultado.

<u>NOTA</u>: se recomienda realizar varias veces las simulaciones en cada ejercicio, ya que éstas son aleatorias, y comentar el resultado más frecuente, o una media de estos.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 5.1 - Análisis de las políticas obtenidas</strong> (0,5 puntos)</div> 

Realizar un estudio de las políticas obtenidas respondiendo a las siguientes preguntas:
- ¿Todos los algoritmos consiguen llegar a la política óptima?
- ¿A qué pueden ser debidas las diferencias entre políticas? (Si las hay)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>CONCLUSIONES:</strong>
<br><br>
Conclusiones aquí
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 5.2 - Influencia del número de episodios</strong> (0,5 puntos)</div> 

Realizar un estudio cambiando el número de episodios en cada uno de los 3 algoritmos indicando:
- Cómo influye el número de episodios en cada algoritmo (a partir de qué número de episodios converge cada algoritmo)
- Comparativa de velocidad entre los 3 algoritmos. ¿Qué método es el más rápido en converger? ¿Cuál es el más lento? ¿A qué es debido?

In [36]:
# Escribir el código de las simulaciones aquí


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>CONCLUSIONES:</strong>
<br><br>
Conclusiones aquí
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 5.3 - Influencia del factor de descuento (discount factor)</strong> (0,5 puntos)</div> 

Ejecutar los 3 algoritmos con *discount factor*=0.1 y el resto de parámetros iguales que en los ejercicios 2, 3 y 4. Describir los cambios en la política óptima, comparando el resultado obtenido con el resultado de los ejercicios 2, 3 y 4 (*discount factor*=1).


DP:

In [49]:
# Escribir el código de las simulaciones aquí
V,policy,i = dp_iteration_value(discount=0.1)

In [50]:
print_policy_DP(policy,11,5)

----------------------
D|D|D|D|D|D|D|D|D|R|U|
----------------------
U|R|R|R|R|R|R|R|R|R|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------
U|U|U|U|U|U|U|U|U|U|U|
----------------------


In [59]:
Q, policy = mc_control_on_policy_epsilon_greedy(env, num_episodes=1670, discount=0.1)

Episode 1670/1670.

In [60]:
print_policy_MC(policy,11,5)

------------------------------------------------------------------------------------------
  U  |  U  |  D  |  D  |  D  |  R  |  U  |  L  |  R  |  R  |  L  |
------------------------------------------------------------------------------------------
  L  |  U  |  U  |  R  |  R  |  D  |  D  |  L  |  U  |  L  |  U  |
------------------------------------------------------------------------------------------
  R  |  D  |  U  |  L  |  L  |  U  |  D  |  U  |  D  |  R  |  U  |
------------------------------------------------------------------------------------------
  D  |  L  |  R  |  L  |  U  |  L  |  D  |  L  |  D  |  R  |  L  |
------------------------------------------------------------------------------------------
  L  |  D  |  U  |  U  |  U  |  R  |  D  |  L  |  U  |  D  |  L  |


In [61]:
Q, policy = Q, policy = mc_control_on_policy_epsilon_greedy(env, num_episodes=1670)

Episode 1670/1670.

In [62]:
print_policy_MC(policy,11,5)

------------------------------------------------------------------------------------------
  D  |  U  |  L  |  U  |  R  |  D  |  R  |  R  |  U  |  U  |  U  |
------------------------------------------------------------------------------------------
  D  |  R  |  U  |  D  |  D  |  R  |  L  |  R  |  R  |  R  |  R  |
------------------------------------------------------------------------------------------
  R  |  D  |  R  |  L  |  R  |  D  |  R  |  R  |  R  |  R  |  U  |
------------------------------------------------------------------------------------------
  D  |  R  |  R  |  R  |  R  |  R  |  U  |  D  |  R  |  R  |  U  |
------------------------------------------------------------------------------------------
  R  |  U  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |  U  |


In [49]:
Q = sarsa_control_algorithm(discount=0.1)

Episode 1000/1000.

In [53]:
policy = {}
V = {}
print_policy_SARSA(Q,11,5)

-------------------------------------------------------------------
  D  |  U  |  U  |  D  |  D  |  D  |  R  |  D  |  D  |  L  |  U  |
-------------------------------------------------------------------
  D  |  L  |  D  |  D  |  D  |  D  |  D  |  R  |  R  |  R  |  U  |
-------------------------------------------------------------------
  D  |  D  |  R  |  D  |  R  |  R  |  D  |  L  |  D  |  R  |  U  |
-------------------------------------------------------------------
  R  |  D  |  R  |  R  |  D  |  R  |  R  |  D  |  D  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  U  |  L  |  R  |  R  |  D  |  R  |  R  |  R  |  U  |


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>CONCLUSIONES:</strong>
<br><br>
Conclusiones aquí
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio 5.4 - Influencia del learning rate</strong> (0,5 puntos)</div> 

Ejecutar el algoritmo de *SARSA* con los siguientes valores de *learning rate*: 0.1 y 0.9. Analizar las diferencias con los resultados obtenidos en la pregunta 4 en términos de número de *time-steps* para llegar a la posición objetivo. Para ello, es necesario usar el mismo número de episodios. Finalmente, comparar el número de pasos por episodio (representar gráficamente el número de *time-steps* para cada episodio de la simulación en ambos casos) y verificar con que valor de *learning rate* el algoritmo converge más rápidamente.

In [54]:
Q = sarsa_control_algorithm(learning_rate=0.1)

Episode 1000/1000.

In [55]:
policy = {}
V = {}
print_policy_SARSA(Q,11,5)

-------------------------------------------------------------------
  D  |  U  |  L  |  U  |  L  |  D  |  U  |  D  |  L  |  D  |  U  |
-------------------------------------------------------------------
  D  |  D  |  R  |  D  |  R  |  R  |  D  |  D  |  D  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  U  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  U  |


In [56]:
Q = sarsa_control_algorithm(learning_rate=0.9)

Episode 1000/1000.

In [57]:
policy = {}
V = {}
print_policy_SARSA(Q,11,5)

-------------------------------------------------------------------
  D  |  L  |  L  |  U  |  D  |  R  |  L  |  U  |  L  |  L  |  U  |
-------------------------------------------------------------------
  R  |  D  |  D  |  L  |  L  |  D  |  L  |  L  |  D  |  R  |  U  |
-------------------------------------------------------------------
  L  |  R  |  D  |  D  |  R  |  R  |  D  |  D  |  D  |  R  |  U  |
-------------------------------------------------------------------
  R  |  R  |  R  |  R  |  U  |  L  |  R  |  D  |  R  |  R  |  U  |
-------------------------------------------------------------------
  D  |  D  |  L  |  U  |  U  |  R  |  R  |  R  |  U  |  U  |  U  |


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>CONCLUSIONES:</strong>
<br><br>
Conclusiones aquí
</div>